## Mounting Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Getting style and content images

In [3]:
from tensorflow import keras

base_image_path = "/content/drive/MyDrive/Colab Notebooks/Los-Angeles-downtown.jpg"
style_reference_image_path = "/content/drive/MyDrive/Colab Notebooks/night1.jpg"

In [4]:
original_width, original_height = keras.utils.load_img(base_image_path).size

img_height = 400
img_width = round(original_width * img_height / original_height)

## Auxiliary Functions

In [5]:
import numpy as np
import tensorflow as tf

def preprocess_image(image_path):
  img = keras.utils.load_img(image_path, target_size=(img_height, img_width))
  img = keras.utils.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = keras.applications.vgg19.preprocess_input(img)
  return img

In [6]:
def deprocess_image(img):
  img = img.reshape((img_height, img_width, 3))
  img[:, :, 0] += 103.939
  img[:, :, 1] += 116.779
  img[:, :, 2] += 123.68
  img = img[:, :, ::-1] # converts from BGR to RGB
  img = np.clip(img, 0, 255).astype("uint8")
  return img

**Using a pretrained VGG19 model to create a feature extractor**

In [7]:
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

80134624/80134624 [==============================] - 3s 0us/step


## Loss function

- `Content Loss`:
  L2 norm between the activations of an upper layer in a pretrained convnet, computed over the target image, and the activations of the same layer computed over the generated image

In [8]:
def content_loss(base_img, combination_img):
  return tf.reduce_sum(tf.square(combination_img - base_img))

- `Style Loss`: Next is the style loss. It uses an auxiliary function to compute the Gram matrix of an
input matrix: a map of the correlations found in the original feature matrix.


![Screenshot from 05-12-23 17:47:41](https://github.com/surajkarki66/MediLeaf_backend/assets/50628520/37fb42b7-0f21-47cd-beaf-76393ed6d0ee)

![Screenshot from 05-12-23 17:56:40](https://github.com/surajkarki66/MediLeaf_backend/assets/50628520/9e8fbdee-88f5-470e-af53-4a028ee4fb4f)


In [9]:
def gram_matrix(x):
  x = tf.transpose(x, (2, 0, 1))
  features = tf.reshape(x, (tf.shape(x)[0], -1))
  gram = tf.matmul(features, tf.transpose(features))
  return gram

In [10]:
def style_loss(style_img, combination_img):
  S = gram_matrix(style_img)
  C = gram_matrix(combination_img)
  channels = 3
  size = img_height * img_width
  return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

- `Total Variational Loss`: To these two loss components, you add a third: the total variation loss, which operates
on the pixels of the generated combination image. It encourages spatial continuity in the generated image, thus avoiding overly pixelated results. You can interpret it as a regularization loss.

In [11]:
def total_variation_loss(x):
  a = tf.square(x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :])
  b = tf.square(x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :])
  return tf.reduce_sum(tf.pow(a + b, 1.25))

The loss that you minimize is a weighted average of these three losses. To compute the content loss, you use only one upper layer—the block5_conv2 layer—whereas for the style loss, you use a list of layers that spans both low-level and high-level layers. You add the total variation loss at the end.

Depending on the style-reference image and content image you’re using, you’ll likely want to tune the content_weight coefficient (the contribution of the content loss to the total loss). A higher content_weight means the target content will be more recognizable in the generated image.

In [12]:
# List of layers to use for the style loss
style_layer_names = [
  "block1_conv1",
  "block2_conv1",
  "block3_conv1",
  "block4_conv1",
  "block5_conv1",
]

# The layer to use for content loss
content_layer_name = "block5_conv2"

# Contribution weight of the total variation loss
total_variation_weight = 1e-6

# Contribution weight of the  style loss
style_weight = 1e-6

# Contribution weight of the  content loss
content_weight = 2.5e-8

In [13]:
def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
          style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

**Setting up the gradient-descent process**

In [14]:
import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))
iterations = 10000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print(f"Iteration {i}: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        keras.utils.save_img(fname, img)

Iteration 100: loss=12281.91
Iteration 200: loss=9658.75
Iteration 300: loss=8654.45
Iteration 400: loss=8104.18
Iteration 500: loss=7746.04
Iteration 600: loss=7489.56
Iteration 700: loss=7295.59
Iteration 800: loss=7142.16
Iteration 900: loss=7016.88
Iteration 1000: loss=6912.29
Iteration 1100: loss=6823.41
Iteration 1200: loss=6746.81
Iteration 1300: loss=6680.04
Iteration 1400: loss=6621.24
Iteration 1500: loss=6568.90
Iteration 1600: loss=6522.08
Iteration 1700: loss=6479.89
Iteration 1800: loss=6441.75
Iteration 1900: loss=6407.10
Iteration 2000: loss=6375.43
Iteration 2100: loss=6346.39
Iteration 2200: loss=6319.72
Iteration 2300: loss=6295.10
Iteration 2400: loss=6272.32
Iteration 2500: loss=6251.19
Iteration 2600: loss=6231.61
Iteration 2700: loss=6213.39
Iteration 2800: loss=6196.37
Iteration 2900: loss=6180.46
Iteration 3000: loss=6165.58
Iteration 3100: loss=6151.68
Iteration 3200: loss=6138.68
Iteration 3300: loss=6126.50
Iteration 3400: loss=6115.06
Iteration 3500: loss=6

In [15]:
!ls

combination_image_at_iteration_10000.png  combination_image_at_iteration_5500.png
combination_image_at_iteration_1000.png   combination_image_at_iteration_5600.png
combination_image_at_iteration_100.png	  combination_image_at_iteration_5700.png
combination_image_at_iteration_1100.png   combination_image_at_iteration_5800.png
combination_image_at_iteration_1200.png   combination_image_at_iteration_5900.png
combination_image_at_iteration_1300.png   combination_image_at_iteration_6000.png
combination_image_at_iteration_1400.png   combination_image_at_iteration_600.png
combination_image_at_iteration_1500.png   combination_image_at_iteration_6100.png
combination_image_at_iteration_1600.png   combination_image_at_iteration_6200.png
combination_image_at_iteration_1700.png   combination_image_at_iteration_6300.png
combination_image_at_iteration_1800.png   combination_image_at_iteration_6400.png
combination_image_at_iteration_1900.png   combination_image_at_iteration_6500.png
combination_image_